# Código para el Productor

In [ ]:
from kafka import KafkaProducer
import json
import random
import time

# Configuración del servidor Kafka
KAFKA_SERVER = 'lab9.alumchat.lol:9092'
TOPIC = '12345'  # Cambia esto por tu número de carné

# Inicialización del productor Kafka
producer = KafkaProducer(
    bootstrap_servers=KAFKA_SERVER,
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

# Función para generar datos simulados de sensores
def generar_datos():
    temperatura = round(random.uniform(0, 110), 2)
    humedad = random.randint(0, 100)
    direccion_viento = random.choice(["N", "NO", "O", "SO", "S", "SE", "E", "NE"])
    return {
        "temperatura": temperatura,
        "humedad": humedad,
        "direccion_viento": direccion_viento
    }

# Enviar datos cada 15-30 segundos
try:
    while True:
        data = generar_datos()
        producer.send(TOPIC, value=data)
        print(f"Datos enviados: {data}")
        time.sleep(random.randint(15, 30))
except KeyboardInterrupt:
    print("Producción interrumpida")
finally:
    producer.close()


# Código para el Consumidor

In [ ]:
from kafka import KafkaConsumer
import json
import matplotlib.pyplot as plt

# Configuración del servidor Kafka
consumer = KafkaConsumer(
    '12345',  # Cambia esto por tu número de carné
    bootstrap_servers='lab9.alumchat.lol:9092',
    value_deserializer=lambda m: json.loads(m.decode('utf-8'))
)

# Inicialización de listas para graficar
temperaturas = []
humedades = []
direcciones_viento = []

# Configuración de la gráfica
plt.ion()
fig, ax = plt.subplots(3, 1, figsize=(10, 8))

def actualizar_graficas():
    ax[0].cla()
    ax[1].cla()
    ax[2].cla()
    ax[0].plot(temperaturas, label="Temperatura (°C)")
    ax[1].plot(humedades, label="Humedad (%)")
    ax[2].plot(direcciones_viento, label="Dirección del Viento")
    ax[0].legend()
    ax[1].legend()
    ax[2].legend()
    plt.pause(0.1)

try:
    for mensaje in consumer:
        datos = mensaje.value
        temperaturas.append(datos["temperatura"])
        humedades.append(datos["humedad"])
        direcciones_viento.append(datos["direccion_viento"])
        actualizar_graficas()
except KeyboardInterrupt:
    print("Consumo interrumpido")
finally:
    plt.ioff()
    plt.show()
